# Computational Physics Assignment

In [ ]:
import numpy as np
from scipy import special
import matplotlib.pyplot as plt
import time

%matplotlib inline

## Q1
#### Floating Point Variables

<p><b>(a)</b> Write a short program to calculate the floating-point machine accuracy on your system.</p>

In [ ]:
# Define machine epsilon of default float function
eps = np.finfo(float).eps

def machine_epsilon(fn=float):
    """
    Compute the machine epsilon.

    This function computes the machine epsilon for the floating point precision
    function passed as an argument.

    Parameters
    ----------
    fn : function, optional
        A generator function which returns a floating point number. Defaults to
        the built-in float function in Python.

    Returns
    -------
    epsilon : scalar
        An estimate of the machine epsilon.

    Raises
    ------
    ValueError
        If `fn` is not a callable function.
        
        If `fn` does not return a `float`, `np.float32`, `np.float64` or
        `np.longdouble` object.

    Notes
    -----
    Machine epsilon is defined as the difference between 1.0 and the next
    largest number which can be represented as a machine-precision number. The
    function initiates with a guess for epsilon (epsilon = 1.0). The test for
    machine accuracy is to add the value of epsilon to 1.0 and verify if the
    computer is able to perform the addition (i.e. if (1.0 + epsilon) is not
    equal to 1.0). If the addition was successful, epsilon is larger than the
    machine accuracy so we reduce epsilon and calculate again.

    References
    ----------
    https://reference.wolfram.com/language/ref/$MachineEpsilon.html

    Examples
    --------
    The following example uses the built-in float function to generate machine
    epsilon. This defaults to a 64-bit floating point number.
    
    >>> machine_epsilon()
    2.220446049250313e-16

    To calculate the machine epsilon using a different precision, pass the
    floating point generator function to the algorithm.

    >>> import numpy as np
    >>> machine_epsilon(np.float32)
    1.1920929e-07

    """
    
    # Create a list of float functions which can be passed to the function
    permitted_fns = [float, np.float32, np.float64, np.longdouble]
    
    # Raise ValueError if the argument is not a function
    if callable(fn) == False:
        raise ValueError('Argument `fn` must be a function.')
    
    # Raise ValueError if the argument does not generate a float
    if type(fn(1)) not in permitted_fns:
        raise ValueError('Argument `fn` must be a float generator function.')
    
    # Declare initial guess for epsilon and declare divisor = 2
    epsilon = fn(1)
    divisor = fn(2)
    
    # Repeatedly reduce epsilon until the addition is unsuccessful
    while fn(1) + (epsilon / divisor) > fn(1):
        epsilon = fn(epsilon) / fn(divisor)

    return epsilon

In [ ]:
print('Default 64-bit float: ', machine_epsilon())

<p><b>(b)</b> What is the machine accuracy of your chosen hardware and programming language for single, double and extended precision floating point variables? What would you expect them to be, theoretically?</p>

In [ ]:
# Single precision float (32-bit)
print('32-bit float: ', machine_epsilon(np.float32))

# Double precision float (64-bit)
print('64-bit float: ', machine_epsilon(np.float64))

# Long double (80-bit on x86 machines, 64-bit otherwise)
print('Long-double: ', machine_epsilon(np.longdouble))

print('Real ME: ', eps)

## Q2
#### Matrix Methods

<p><b>(a)</b> Write code to carry out LU decomposition of an arbitrary $N \times N$ matrix using Crout’s method. Return the result in the form of an $N \times N$ matrix containing all the elements of $U$, and the non-diagonal elements of $L$. Your routine may either return the result as a new matrix, or just overwrite the input matrix. You needn’t implement pivoting (although you can if you like).</p>

In [ ]:
# Define machine epsilon
eps = np.finfo(float).eps

def crout_LU(mat):
    """
    Decompose a NxN matrix using Crout's method.

    This function decomposes a NxN matrix into a lower triangular matrix (L)
    and an upper triangular matrix (U) using Crout's method. The L matrix is
    assumed to have 1's along its diagonals. The returned result is an NxN
    matrix which combines the L and U matrices.

    Parameters
    ----------
    mat : array_like
        Input matrix expressed as an NxN array of scalar values.

    Returns
    -------
    mat : ndarray
        An NxN matrix combining the upper (U) and lower (L) triangular
        matrices.

    Raises
    ------
    ValueError
        If any of the elements in the input matrix cannot be parsed into a
        float.
        
        If the input matrix is not a square matrix.
        
        If the upper matrix contains a zero along its diagonals (since pivoting
        has not been implemented).

    See Also
    --------
    get_lu_matrix : Returns the L and U matrices from crout_LU().
    solve_matrix_eqn : Solves a decomposed LU matrix equation.

    Notes
    -----
    A matrix equation A.x = b can be solved more easily if the matrix A can be
    decomposed into a lower triangular matrix (L) and an upper triangular
    matrix (U) (i.e. A = L.U). The system then reduces to L.U.x = b which can
    be solved by forward and backward substitution.

    References
    ----------
    Numerical Recipes in C, Crout's method

    Examples
    --------
    >>> A = [[1, 2], [3, 4]]
    >>> crout_LU(A)
    array( [[ 1.  2.]
            [ 3. -2.]] )

    Use this method to solve a linear system of equations defined by A.x = b
    
    >>> A = [[1, 2], [3, 4]]
    >>> b = [1, 1]
    >>> (l, u, det) = get_lu_matrix(crout_LU(A))
    >>> solve_matrix_eqn(l, u, b)
    array( [-1.  1.] )

    """
    
    # Attempt to parse as a Numpy array
    mat = np.array(mat, dtype=float)

    (row, col) = np.shape(mat)
    
    # Verify the input matrix is a square matrix
    if row != col:
        raise ValueError('Argument `mat` must be a square matrix.')
    
    # Loop through each column
    for j in range(col):
        
        # For each column, go down to the diagonal
        for i in range(j + 1):
            # Update the input matrix to the values for the upper matrix
            mat[i, j] -= sum([mat[i, k] * mat[k, j] for k in range(i)])
            
            # Crout's algorithm is unstable if the diagonal contains a zero
            # Raise a ValueError if this occurs
            if np.abs(mat[j, j]) < eps:
                raise ValueError('Crout\'s method is unstable with the ' +
                                 'argument `mat` provided.')
        
        # For each column, loop through the elements below the diagonal
        for i in range(j + 1, col):            
            # Update the input matrix to the values for the lower matrix
            s = sum([mat[i, k] * mat[k, j] for k in range(j)])
            mat[i, j] = (mat[i, j] - s) / mat[j, j]
    
    # Return the updated matrix
    return mat

def get_lu_matrix(a):
    """
    Reconstruct the L and U matrices from the decomposed matrix.

    Given a decomposed matrix from the crout_LU() function, this function
    reconstructs the lower triangular (L) and upper triangular (U) matrices.

    Parameters
    ----------
    a : array_like
        An NxN decomposed matrix as returned by crout_LU().

    Returns
    -------
    A tuple containing:
    
    l : ndarray
        An NxN lower triangular (L) matrix with ones on its diagonal.
    
    u : ndarray
        An NxN upper triangular matrix (U).
    
    det : scalar
        The determinant of the matrix A = L.U.

    Raises
    ------
    ValueError
        If any of the elements in the input matrix cannot be parsed into a
        float.
        
        If the input matrix is not a square matrix.

    See Also
    --------
    crout_LU : Returns a decomposed matrix.
    solve_matrix_eqn : Solves a decomposed LU matrix equation.

    Notes
    -----
    The steps involved in crout_LU() are computationally intensive, so we avoid
    constructing the LU matrix from scratch. We use the decomposed matrix from
    crout_LU() to find the L and U matrices. In this representation, we can
    also find the determinant cheaply by taking a product of the diagonal
    elements of the decomposed matrix.

    References
    ----------
    Numerical Recipes in C, Crout's method

    Examples
    --------
    Use this method to solve a linear system of equations defined by A.x = b
    
    >>> A = [[1, 2], [3, 4]]
    >>> b = [1, 1]
    >>> get_lu_matrix(crout_LU(A))
    (
    array( [[1., 0.],
            [3., 1.]]),
    array( [[ 1.,  2.],
            [ 0., -2.]]),
    -2.0
    )
    
    Get the determinant of a matrix
    
    >>> A = [[3, 1, 0, 0, 0], [3, 9, 4, 0, 0], [0, 9, 20, 10, 0],
             [0, 0, -22, 31, -25], [0, 0, 0, -55, 61]]
    >>> (l, u, det) = get_lu_matrix(crout_LU(A))
    >>> det
    514032.0

    """
    
    # Attempt to parse as a Numpy array
    a = np.array(a, dtype=float)

    (row, col) = np.shape(a)
    
    # Verify the input matrix is a square matrix
    if row != col:
        raise ValueError('Argument `a` must be a square matrix.')
    
    # Construct template L and U matrices
    l = np.identity(row)
    u = np.zeros((row, col))
    det = 1.0
    
    # Loop through each row of the decomposed matrix
    for i in range(row):
        
        # Loop through each column of the decomposed matrix
        for j in range(col):
            
            if i <= j:
                # Update elements of the upper matrix
                u[i, j] = a[i, j]
            if j < i:
                # Update elements of the lower matrix
                l[i, j] = a[i, j]
            if i == j:
                # The determinant is the product of the diagonal elements
                det *= a[i, j]
    
    # Return a tuple containing the LU matrices and the determinant
    return (l, u, det)

<p><b>(b)</b> Use your routine to express the matrix $$A = \left[ {\begin{array}{ccccc} 3 & 1 & 0 & 0 & 0 \\ 3 & 9 & 4 & 0 & 0 \\ 0 & 9 & 20 & 10 & 0 \\ 0 & 0 & -22 & 31 & -25 \\ 0 & 0 & 0 & -55 & 61\end{array}} \right]$$ as a product of upper and lower diagonal matrices, and compute $\mathrm{det}(A)$ using your answer.</p>

In [ ]:
A = [[3, 1, 0, 0, 0],
     [3, 9, 4, 0, 0],
     [0, 9, 20, 10, 0],
     [0, 0, -22, 31, -25],
     [0, 0, 0, -55, 61]]
(l, u, det) = get_lu_matrix(crout_LU(A))
print('Lower matrix: \n', l)
print('Upper matrix: \n', u)
print('Determinant: ', det)
print('Product of LU should give A: \n', np.matmul(l, u))

<p><b>(c)</b> Write a short function that solves the matrix equation $LU\mathrm{\textbf{x}}=\mathrm{\textbf{b}}$ using forward and backward substitution where $L$ and $U$ are $N \times N$ upper and lower-triangular matrices, respectively. Here, $L$, $U$, and the vector $\mathrm{\textbf{b}}$ should be input parameters of your routine.</p>

In [ ]:
def solve_matrix_eqn(l, u, b):
    """
    Solves the matrix equation L.U.x = b.
    
    Given a lower triangular matrix `l`, an upper triangular matrix `u` and a
    solution vector `b`, this function finds the vector `x` such that
    L.U.x = b.

    Parameters
    ----------
    l : array_like
        An NxN lower triangular matrix with ones on its diagonals.
    
    u : array_like
        An NxN upper triangular matrix.
    
    b : array_like
        An N-dimensional vector.

    Returns
    -------
    x : ndarray
        An N-dimensional vector which solves the equation L.U.x = b.

    Raises
    ------
    ValueError
        If any of the elements in the input matrices cannot be parsed into
        floats.
        
        If `l` and `u` are not NxN matrices.
        
        If the length of `b` is not N.

    See Also
    --------
    crout_LU : Returns a decomposed matrix.
    solve_matrix_eqn : Solves a decomposed LU matrix equation.

    Notes
    -----
    The steps involved in crout_LU() are computationally intensive, so we avoid
    constructing the LU matrix from scratch. We use the decomposed matrix from
    crout_LU() to find the L and U matrices. In this representation, we can
    also find the determinant cheaply by taking a product of the diagonal
    elements of the decomposed matrix.

    References
    ----------
    Numerical Recipes in C, Crout's method

    Examples
    --------
    >>> A = [[1, 2], [3, 4]]
    >>> crout_LU(A)
    array( [[ 1.  2.]
            [ 3. -2.]] )

    Use this method to solve a linear system of equations defined by A.x = b
    
    >>> A = [[1, 2], [3, 4]]
    >>> b = [1, 1]
    >>> get_lu_matrix(crout_LU(A))
    (
    array( [[1., 0.],
            [3., 1.]]),
    array( [[ 1.,  2.],
            [ 0., -2.]]),
    -2.0
    )

    """
    
    # Argument validation
    l = np.array(l, dtype=float)

    (l_row, l_col) = np.shape(l)
    if l_row != l_col:
        raise ValueError('Argument `l` must be a square matrix.')

    u = np.array(u, dtype=float)

    (u_row, u_col) = np.shape(u)
    if u_row != u_col:
        raise ValueError('Argument `u` must be a square matrix.')

    if l_row != u_row:
        raise ValueError('Arguments `l` and `u` must have the same ' +
                         'dimensions.')

    b = np.array(b, dtype=float)

    b_row = np.shape(b)
    if b_row[0] != l_row:
        raise ValueError('Argument `b` must be the same length as arguments ' +
                         '`l` and `u`.')
    
    # Loop through each element
    for i in range(l_row):
        # Use forward substitution to update the values of b -> y
        s = sum([l[i, j] * b[j] for j in range(i)])
        b[i] = b[i] - s
    for i in range(l_row - 1, -1, -1):
        # Use backward substitution to update the values of y -> x
        if np.abs(u[i, i]) < eps:
            # Prevent division by zero error
            raise ValueError('Argument `u` cannot contain a zero on its ' +
                             'diagonal.')
        else:
            s = sum([u[i, j] * b[j] for j in range(i + 1, l_row)])
            b[i] = (b[i] - s) / u[i, i]
    
    # Return the solution vector x
    return b

<p><b>(d)</b> Use your routine together with that of <b>(a)</b> to solve $$A\mathrm{\textbf{x}}=\left[ {\begin{array}{c} 2 \\ 5 \\ -4 \\ 8 \\ 9 \end{array}} \right]$$ for $\mathrm{\textbf{x}}$, where $A$ is as per <b>(b)</b>.</p>

In [ ]:
A = [[3, 1, 0, 0, 0],
     [3, 9, 4, 0, 0],
     [0, 9, 20, 10, 0],
     [0, 0, -22, 31, -25],
     [0, 0, 0, -55, 61]]
b = [2, 5, -4, 8, 9]
(l, u, det) = get_lu_matrix(crout_LU(A))
print('x = ', solve_matrix_eqn(l, u, b))

<p><b>(e)</b> Use your routine together with that of <b>(a)</b> to calculate $A^{-1}$.</p>

In [ ]:
def invert(mat):
    """
    Invert an NxN matrix using LU decomposition.

    This function finds the inverse of an NxN matrix by performing LU
    decomposition and solving each column with the corresponding column from
    the identity matrix.

    Parameters
    ----------
    mat : array_like
        Input matrix expressed as an NxN array of scalar values.

    Returns
    -------
    inv : ndarray
        An N-dimensional vector which is the inverse of the input matrix.

    Raises
    ------
    ValueError
        If any of the elements in the input matrices cannot be parsed into
        floats.
        
        If `l` and `u` are not NxN matrices.
        
        If the length of `b` is not N.

    See Also
    --------
    crout_LU : Returns a decomposed matrix.
    get_lu_matrix : Gets the LU matrices from a decomposed matrix.
    solve_matrix_eqn : Solves a decomposed LU matrix equation.

    Notes
    -----
    We decompose the input matrix into its L and U compositions using Crout's
    method. We equate this to the identity matrix and solve for each vector x
    which solves the equation A.x = I where I is the corresponding column of
    the identity matrix.

    References
    ----------
    Numerical Recipes in C, Crout's method

    Examples
    --------
    >>> A = [[1, 2], [3, 4]]
    >>> invert(A)
    array( [[-2.   1. ]
            [ 1.5 -0.5]] )

    """
    
    # Attempt to parse as a numpy array
    mat = np.array(mat, dtype=float)

    (row, col) = np.shape(mat)
    
    # Verify the input matrix is a square matrix
    if row != col:
        raise ValueError('Argument `mat` must be a square matrix.')
    
    # Construct an identity matrix which has the same size as the input matrix
    identity = np.identity(col)
    
    # Decompose the input matrix
    (l, u, det) = get_lu_matrix(crout_LU(mat))
    
    # Solve the equation for each column of the identity matrix
    cols = [solve_matrix_eqn(l, u, identity[:, i]) for i in range(col)]
    inv = np.column_stack(cols)  # Stack the columns into a matrix
    return inv

In [ ]:
A = [[3, 1, 0, 0, 0],
     [3, 9, 4, 0, 0],
     [0, 9, 20, 10, 0],
     [0, 0, -22, 31, -25],
     [0, 0, 0, -55, 61]]
invA = invert(A)
print('Inverse of A: ', invA)
print('A * invA = identity matrix: ', np.matmul(A, invA))

## Q3
#### Interpolation

<p><b>(a)</b> Write a simple routine to perform linear interpolation on a tabulated set of $x$-$y$ data.</p>

In [ ]:
# Define machine epsilon
eps = np.finfo(float).eps

def linear_interp(data):
    """
    Perform a linear interpolation of data.

    This function performs a first order interpolation on a tabulated set of
    data.

    Parameters
    ----------
    data : array_like
        A matrix of tabulated data containing a column each for the independent
        and dependent variables.

    Returns
    -------
    interp : function
        A function which takes one numeric argument and returns the
        interpolated value at that point.

    Raises
    ------
    ValueError
        If any of the elements in the input matrix cannot be parsed into a
        float.
        
        If the input matrix does not contain a column for the independent
        variable and a column for the dependent variable.

    See Also
    --------
    cubic_spline : Performs cubic spline interpolation.

    Notes
    -----
    A linear interpolation can be performed on a set of data (x, y) by noting
    that we can write f(x) = A(x)y0 + B(x)y1 and compute expressions for A(x)
    and B(x) from the data.

    References
    ----------
    Numerical Recipes in C, Interpolation

    Examples
    --------
    >>> Ax = [-2.1, -1.45, -1.3, -0.2, 0.1, 0.15, 0.9, 1.1, 1.5, 2.8, 3.8]
    >>> Ay = [0.012155, 0.122151, 0.184520, 0.960789, 0.990050, 0.977751,
              0.422383, 0.298197, 0.105399, 3.936690e-4, 5.355348e-7]
    >>> A = np.column_stack((Ax, Ay))
    >>> lin_int = linear_interp(A)
    >>> lin_int(1)
    0.36029

    """
    
    # Attempt to parse as a Numpy array
    data = np.array(data, dtype=float)
    
    # Ensure are columns for independent and dependent variables
    (d_row, d_col) = np.shape(data)
    if d_col != 2:
        raise ValueError('Argument `data` must be tabulated.')

    # Sort the data according to the x values
    # This step is computationally expensive
    # However, sorted data is more efficient to work with
    data = data[np.argsort(data[:, 0])]
    
    # Define the function which will perform the interpolation
    def interp(x):
        # For an input x, we find the value of the index i such that
        # data[i - 1] < x <= data[i]
        i = np.searchsorted(data[:, 0], x)
        
        # If x is greater than the domain of the data, raise an error
        if i == len(data):
            raise ValueError('Argument `x` is outside the range of ' +
                             'interpolation.')
        
        # If x is equal to one of the data points, no interpolation is required
        if np.abs(data[i, 0] - x) < eps:
            return data[i, 1]
        
        # If x is less than the domain of the data, raise an error
        if i == 0:
            raise ValueError('Argument `x` is outside the range of ' +
                             'interpolation.')
        
        # Calculate the values for A(x) and B(x)
        a = (data[i, 0] - x) / (data[i, 0] - data[i - 1, 0])
        b = (x - data[i - 1, 0]) / (data[i, 0] - data[i - 1, 0])
        
        # Calculate and return the interpolated value
        interp_x = (a * data[i - 1, 1]) + (b * data[i, 1])
        return interp_x
    
    # Return the interpolation function
    return interp

<p><b>(b)</b> Write a routine to perform cubic spline interpolation on a tabulated set of $x$-$y$ data, using the natural spline boundary condition. The matrix solver should be called as an external routine rather than coded inline into the interpolator like in Numerical Recipes. Use your own matrix solver from Q2 for this.</p>

In [ ]:
def cubic_spline(data):
    """
    Perform a cubic spline interpolation of data.

    This function performs a third order interpolation on a tabulated set of
    data using the natural spline (second derivatives zero at boundaries).

    Parameters
    ----------
    data : array_like
        A matrix of tabulated data containing a column each for the independent
        and dependent variables.

    Returns
    -------
    interp : function
        A function which takes one numeric argument and returns the
        interpolated value at that point.

    Raises
    ------
    ValueError
        If any of the elements in the input matrix cannot be parsed into a
        float.
        
        If the input matrix does not contain a column for the independent
        variable and a column for the dependent variable.

    See Also
    --------
    linear_interp : Performs cubic spline interpolation.

    Notes
    -----
    A cubic spline interpolation can be performed on a set of data (x, y) by
    noting that we can write f(x) = A(x)y0 + B(x)y1 + C(x)y0'' + D(x)y1'' and
    compute expressions for A(x), B(x), C(x), D(x) from the data. Usually, a
    matrix solving technique is required.

    References
    ----------
    Numerical Recipes in C, Interpolation

    Examples
    --------
    >>> Ax = [-2.1, -1.45, -1.3, -0.2, 0.1, 0.15, 0.9, 1.1, 1.5, 2.8, 3.8]
    >>> Ay = [0.012155, 0.122151, 0.184520, 0.960789, 0.990050, 0.977751,
              0.422383, 0.298197, 0.105399, 3.936690e-4, 5.355348e-7]
    >>> A = np.column_stack((Ax, Ay))
    >>> cub_spl = cubic_spline(A)
    >>> cub_spl(1)
    0.3564446319717923

    """
    
    # Attempt to parse as a Numpy array
    data = np.array(data, dtype=float)
    
    # Ensure are columns for independent and dependent variables
    (d_row, d_col) = np.shape(data)
    if d_col != 2:
        raise ValueError('Argument `data` must be tabulated.')

    # Sort the array
    # This step is computationally expensive
    # However, sorted data is more efficient to work with
    data = data[np.argsort(data[:, 0])]
    
    # Define matrix of coefficients for second derivative system of equations
    coeff_mat = np.identity(d_row)
    b_mat = np.zeros((d_row))  # Initiate the derivatives matrix
    
    # We do not loop for i = 0 or i = d_row - 1 since these elements are in the
    # identity matrix and we can directly set their value in the b_matrix = 0
    # to produce the natural spline.
    for i in range(1, d_row - 1):
        x0 = data[i - 1, 0]
        x1 = data[i, 0]
        x2 = data[i + 1, 0]
        
        y0 = data[i - 1, 1]
        y1 = data[i, 1]
        y2 = data[i + 1, 1]
        
        coeff_mat[i, i - 1] = (x1 - x0) / 6
        coeff_mat[i, i] = (x2 - x0) / 3
        coeff_mat[i, i + 1] = (x2 - x1) / 6
        
        b_mat[i] = ((y2 - y1) / (x2 - x1)) - ((y1 - y0) / (x1 - x0))
    
    # Decompose the coefficient matrix
    (l, u, det) = get_lu_matrix(crout_LU(coeff_mat))
    
    # Solve the system of equations to find the second derivatives of y
    y_der = solve_matrix_eqn(l, u, b_mat)
    
    # Define the function which will perform the interpolation
    def interp(x):
        # For an input x, we find the value of the index i such that
        # data[i - 1] < x <= data[i]
        i = np.searchsorted(data[:, 0], x)
        
        # If x is greater than the domain of the data, raise an error
        if i == len(data):
            raise ValueError('Argument `x` is outside the range of ' +
                             'interpolation.')
        
        # If x is equal to one of the data points, no interpolation is required
        if np.abs(data[i, 0] - x) < eps:
            return data[i, 1]
        
        # If x is less than the domain of the data, raise an error
        if i == 0:
            raise ValueError('Argument `x` is outside the range of ' +
                             'interpolation.')
        
        # Calculate the values for A(x), B(x), C(x) and D(x)
        x0 = data[i - 1, 0]
        x1 = data[i, 0]
        
        a = (x1 - x) / (x1 - x0)
        b = (x - x0) / (x1 - x0)
        c = ((a ** 3 - a) * (x1 - x0) ** 2) / 6
        d = ((b ** 3 - b) * (x1 - x0) ** 2) / 6
        
        # Calculate and return the interpolated value
        y0 = data[i - 1, 1]
        y1 = data[i, 1]
        
        interp_x = (a * y0) + (b * y1) + (c * y_der[i - 1]) + (d * y_der[i])
        return interp_x
    
    # Return the interpolation function
    return interp

<p><b>(c)</b> Consider the following table of data
    <table align="center">
        <tr>
            <th style="text-align:center; font-weight: bold">$x$</th>
            <th style="text-align:center; font-weight: bold">$y$</th>
        </tr>
        <tr>
            <td style="text-align:center">$-2.1$</td>
            <td style="text-align:center">$0.012155$</td>
        </tr>
        <tr>
            <td style="text-align:center">$-1.45$</td>
            <td style="text-align:center">$0.122151$</td>
        </tr>
        <tr>
            <td style="text-align:center">$-1.3$</td>
            <td style="text-align:center">$0.184520$</td>
        </tr>
        <tr>
            <td style="text-align:center">$-0.2$</td>
            <td style="text-align:center">$0.960789$</td>
        </tr>
        <tr>
            <td style="text-align:center">$0.1$</td>
            <td style="text-align:center">$0.990050$</td>
        </tr>
        <tr>
            <td style="text-align:center">$0.15$</td>
            <td style="text-align:center">$0.977751$</td>
        </tr>
        <tr>
            <td style="text-align:center">$0.9$</td>
            <td style="text-align:center">$0.422383$</td>
        </tr>
        <tr>
            <td style="text-align:center">$1.1$</td>
            <td style="text-align:center">$0.298197$</td>
        </tr>
        <tr>
            <td style="text-align:center">$1.5$</td>
            <td style="text-align:center">$0.105399$</td>
        </tr>
        <tr>
            <td style="text-align:center">$2.8$</td>
            <td style="text-align:center">$3.936690 \times 10^{-4}$</td>
        </tr>
        <tr>
            <td style="text-align:center">$3.8$</td>
            <td style="text-align:center">$5.355348 \times 10^{-7}$</td>
        </tr>
    </table>
Using both linear interpolation and natural cubic splines, interpolate the tabulated function everywhere between its first and last entries, and plot the results on the same set of axes. Use some high enough density of points that your curves look continuous and smooth (where relevant).</p>

In [ ]:
# Define the data
Ax = [-2.1, -1.45, -1.3, -0.2, 0.1, 0.15, 0.9, 1.1, 1.5, 2.8, 3.8]
Ay = [0.012155, 0.122151, 0.184520, 0.960789, 0.990050, 0.977751, 0.422383,
      0.298197, 0.105399, 3.936690e-4, 5.355348e-7]
A = np.column_stack((Ax, Ay))

# Linear interpolation
fn1 = linear_interp(A)

# Cubic spline interpolation
fn2 = cubic_spline(A)

# Define domain
x = np.linspace(-2.1, 3.8, 200)

# Calculate and plot
y1 = [fn1(i) for i in x]
y2 = [fn2(i) for i in x]
plt.plot(x, y1, label='Linear interpolation')
plt.plot(x, y2, label='Cubic spline')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.grid()
plt.legend()
plt.plot(Ax, Ay, ls='', marker='x', color='black', ms=10)
plt.show()

In [ ]:
# Define the data
Ax = [-2.1, -1.45, -1.3, -0.2, 0.1, 0.15, 0.9, 1.1, 1.5, 2.8, 3.8]
Ay = np.log([0.012155, 0.122151, 0.184520, 0.960789, 0.990050, 0.977751, 0.422383,
      0.298197, 0.105399, 3.936690e-4, 5.355348e-7])
A = np.column_stack((Ax, Ay))

# Linear interpolation
fn3 = linear_interp(A)

# Cubic spline interpolation
fn4 = cubic_spline(A)

# Calculate and plot
y1 = [fn3(i) for i in x]
y2 = [fn4(i) for i in x]
plt.plot(x, y1, label='Linear interpolation')
plt.plot(x, y2, label='Cubic spline')
plt.xlabel('$x$')
plt.ylabel('$\log y$')
plt.grid()
plt.legend()
plt.plot(Ax, Ay, ls='', marker='x', color='black', ms=10)
plt.show()

## Q4
#### Fourier Transforms

<p><b>(a)</b> Write a program that uses either numpy.fft or the external FFT library FFTW (www.fftw.org, available in
Python as package pyFFTW) to convolve the signal function $${\begin{array}{l r} h(t)=4 & \mathrm{for \,} 3 \leq t \leq 5 \\ h(t)=0 & \mathrm{otherwise} \end{array}}$$ with the response function $$g(t)=\frac{1}{\sqrt{2\pi}}e^{-t^2/2}\,,$$ using the spectral techniques described in lectures. Explain your design choices regarding sampling, aliasing and padding.</p>

In [ ]:
# Define machine epsilon
eps = np.finfo(float).eps

# Define the top hat function
def h(t):
    if t < 3 or t > 5:
        return 0
    else:
        return 4

# Define the Gaussian function
def g(t):
    return np.exp(-(t ** 2) / 2) / np.sqrt(2 * np.pi)

def convolution(sig, res, dt):
    """
    Perform a convolution of a signal and a response.

    This function uses fast Fourier transforms and the convolution theorem to
    calculate the convolution of a signal with a response over a given
    interval.

    Parameters
    ----------
    sig : array_like
        A vector sequence which has length of a power of two containing samples
        of the signal with appropriate padding.
    
    res : array_like
        A vector sequence which has the same length as `sig` and contains
        samples of the response function in wrap-around format.
    
    dt : scalar
        The time spacing between samples in `sig` and `res`.

    Returns
    -------
    conv : ndarray
        A vector sequence which contains the discrete convolution of the signal
        with the response.

    Raises
    ------
    ValueError
        If any of the elements in the input vectors cannot be parsed into a
        float.
        
        If `dt` is zero or negative.
        
        If the length of `sig` or `res` is not a power of two.

    Notes
    -----
    The discrete convolution can be performed by using the convolution theorem.
    To perform a convolution, the signal and response sequences are Fourier
    transformed using the FFT routine, then multiplied element-wise. The
    resulting sequence is inverse Fourier transformed.

    References
    ----------
    Numerical Recipes in C, Convolutions using FFTs

    Examples
    --------
    Convolve a top hat signal with a Gaussian function
    >>> (T, dt) = np.linspace(-10, 10, 128, retstep=True)
    >>> sig = [h(t) for t in T]
    >>> res = [g(t) for t in np.arange(0, 3, dt)]
    >>> res.extend([0 for t in range(128 - 2 * len(res))])
    >>> res.extend([g(t) for t in np.arange(-3, 0, dt)])
    >>> conv = convolution(sig, res, dt).real
    >>> plt.plot(T, conv)
    >>> plt.show()

    """
    
    if dt < eps:
        raise ValueError('Sample spacing `dt` is too small.')
    
    if len(sig) == 0 or np.log2(len(sig)).is_integer() == False:
        raise ValueError('The length of `sig` must be a power of two.')
    
    if len(res) == 0 or np.log2(len(res)).is_integer() == False:
        raise ValueError('The length of `res` must be a power of two.')
    
    # Parse the arrays as Numpy float arrays
    sig = np.array(sig, dtype=float)
    res = np.array(res, dtype=float)
    
    # Fast Fourier transform the sequences
    F_signal = np.fft.fft(sig)
    F_response = np.fft.fft(res)
    
    # Perform the element-wise multiplication
    F_conv = (F_signal * F_response)
    
    # Inverse Fourier transform to retrieve the convolution
    # We do not need to divide by N as this is already included
    # in the np.fft.ifft definition.
    conv = np.fft.ifft(F_conv)
    
    # Multiply by the spacing interval
    return dt * conv
    
# Convolution function for the top hat signal and the Gaussian response.
def TH_Gaussian_convolve(start, end, n):
    
    # Define the domain
    (T, dt) = np.linspace(start, end, n, retstep=True)
    
    # Define the signal samples
    signal = [h(t) for t in T]
    
    # Define the response samples in wrap-around format
    response = [g(t) for t in np.arange(0, 3.5, dt)]
    response.extend([0 for t in range(n - 2 * len(response))])
    response.extend([g(t) for t in np.arange(-3.5, 0, dt)]);
    
    conv = convolution(signal, response, dt)
    return (T, conv.real)

<p><b>(b)</b> Plot $h(t)$, $g(t)$ and $(h*g)(t)$ over appropriate ranges in $t$.</p>

In [ ]:
# Define analytic solution
def p(t):
    return 2 * (special.erf((t - 3) / np.sqrt(2)) - special.erf((t - 5) / np.sqrt(2)))

(T, conv) = TH_Gaussian_convolve(-7, 15, 512)
sig = [h(t) for t in T]
res = [g(t) for t in T]
ana = [p(t) for t in T]

plt.plot(T, conv, label='Convolution, (h*g)(t)')
plt.plot(T, sig, label='Signal, h(t)')
plt.plot(T, res, label='Response, g(t)')
plt.plot(T, ana, label='Analytic convolution, p(t)', ls='dashed')
plt.xlim(-5, 8)
plt.xlabel('t')
plt.ylabel('y')
plt.grid()
plt.legend()
plt.show()

In [ ]:
"""
For t < 0, the response sequence should wrap around to the end of
the sequence. To see this behaviour more clearly, consider the plot
below of the Gaussian response function in wrap-around format. Note
that no matter how large the interval is, the function should
wrap-around for negative times.

"""

n = 256

(T, dt) = np.linspace(0, 15, n, retstep=True)
response = [g(t) for t in np.arange(0, 3.5, dt)]
response.extend([0 for t in range(n - 2 * len(response))])
response.extend([g(t) for t in np.arange(-3.5, 0, dt)]);
plt.plot(T, response)
plt.title('Interval = 15')
plt.xlabel('t')
plt.ylabel('y')
plt.grid()
plt.show()

(T, dt) = np.linspace(0, 50, n, retstep=True)
response = [g(t) for t in np.arange(0, 3.5, dt)]
response.extend([0 for t in range(n - 2 * len(response))])
response.extend([g(t) for t in np.arange(-3.5, 0, dt)]);
plt.plot(T, response)
plt.title('Interval = 50')
plt.xlabel('t')
plt.ylabel('y')
plt.grid()
plt.show()

## Q5
#### Random Numbers

<p><b>(a)</b> Using a decent built-in uniform deviate generator (or one from an external library – either way, justify
your choice), write a short program to compute $10^5$ uniformly-distributed random numbers over the interval $x \in \left[ 0, 1 \right]$, based on a single seed. Plot the resultant distribution as a binned histogram, with the value of $x$ on the $x$ axis and the number of samples in the bin on the $y$ axis. Choose an appropriate number of bins.</p>

In [ ]:
# Use a fixed seed
np.random.seed(22092009)

# Generate a sample of uniformly distributed numbers
x = np.random.rand(100000)

# Plot
bins = 16
X = np.linspace(0, 1, 2)
Y = [100000 / bins for x in X]
plt.hist(x, bins, edgecolor='black')
plt.plot(X, Y, label='Expectation Value')
plt.xlabel('Random variate')
plt.ylabel('Samples')
plt.legend()
plt.show()

<p><b>(b)</b> Now compute random numbers distributed over the interval $x \in \left[ 0, \pi \right]$ as $$\mathrm{pdf}(x)=\frac{1}{2}\cos\left(\frac{x}{2}\right)\,.$$ Use the transformation method, and show your preparatory working out. Plot the resultant distribution for $10^5$ points based on a single seed.</p>

In [ ]:
# Define the PDF and CDF for Q5 (b)
def pdf1(x):
    return np.cos(x / 2) / 2

def cdf1(x):
    return np.sin(x / 2)

# Define the inverse CDF function used for the transformation method in Q5 (b)
def inv_cdf1(x):
    return 2 * np.arcsin(x)

# Define a function to calculate the empirical CDF
def ecdf(x):
    """
    Computes the empirical cumulative distribution function for a set of data.

    This function calculates the empirical cumulative distribution function for
    a set of random variables drawn from some probability distribution.

    Parameters
    ----------
    x : array_like
        A vector containing random variables drawn from a probability
        distribution.

    Returns
    -------
    ecdf : tuple
        A tuple containing the x-coordinates and y-coordinates of the empirical
        cumulative distribution function.

    Raises
    ------
    ValueError
        If any of the elements in the input vector cannot be parsed into a
        float.
        
        If the length of the input vector is zero.

    Notes
    -----
    An empirical CDF can be used as a comparison against the CDF from an
    expected distribution. The value of the CDF at a point `x` is the integral
    of the corresponding PDF from -infinity to x. In the empirical case, this
    can be computed by calculating the proportion of values in a vector that
    are less than x.

    References
    ----------
    Numerical Recipes in C, Random numbers

    Examples
    --------
    >>> y = transformationMethod(inv_cdf1, 100000)
    >>> (ecdf_x, ecdf_y) = ecdf(y)
    >>> plt.plot(ecdf_x, ecdf_y)
    >>> plt.show()

    """
    
    # Divide by len(x) later for normalisation so len(x) != 0
    if len(x) == 0:
        raise ValueError('The input vector `x` cannot be of zero length.')
    
    # Parse the array as a Numpy float array
    x = np.array(x, dtype=float)
    
    # Order the random variables from smallest to largest
    xs = np.sort(x)
    
    # Increment the ECDF value by one for each random variable
    # Normalise by dividing by the total number of random variables
    ys = np.arange(1, len(xs) + 1) / len(xs)
    
    # Return the coordinates as a tuple which can be unpacked
    return (xs, ys)

# Define the transformation method
def transformationMethod(inv_cdf, n=1):
    """
    Generate random variables using the transformation method.

    This function generates random variables from a probability distribution
    given its inverse cumulative distribution function.

    Parameters
    ----------
    inv_cdf : function
        A function which returns a number for all values in the range zero to
        one, inclusive.
    
    n : int, optional
        The number of random variables to generate, defaults to one.

    Returns
    -------
    y : list
        A list containing the random variables.

    Raises
    ------
    ValueError
        If `inv_cdf` is not a callable function.
        
        If `inv_cdf` does not return a `float`, `np.float32`, `np.float64` or
        `np.longdouble` object.
        
        If `n` is non-integer or less than one.

    See Also
    --------
    rejectionMethod : Generates random variables using the rejection method.

    Notes
    -----
    A transformation method uses random numbers between [0, 1] to calculate
    random variables from the inverse of its cumulative distribution function.
    This method is usually more efficient than the rejection method, but only
    works in analytic cases where the inverse of the cumulative distribution
    function is know. If only the PDF is known, the rejection method should be
    used.

    References
    ----------
    Numerical Recipes in C, Random numbers

    Examples
    --------    
    Generate a vector of vector of random numbers
    >>> transformationMethod(inv_cdf1, 3)
    [2.0987481837335364, 0.4168466206309118, 1.507681160761436]
    
    """
    
    if (n / 1.0).is_integer() == False or n < 1:
        raise ValueError('Argument `n` must be a positive integer.')
    
    # Create a list of float functions which can be passed to the function
    permitted_fns = [float, np.float32, np.float64, np.longdouble]
    
    # Raise ValueError if the argument is not a function
    if callable(inv_cdf) == False:
        raise ValueError('Argument `inv_cdf` must be a function.')
    
    # Raise ValueError if the argument does not generate a float
    if type(inv_cdf(1)) not in permitted_fns:
        raise ValueError('Argument `inv_cdf` must be a float generator ' +
                         'function.')
    
    
    # Return a list of random numbers
    x = np.random.rand(n)
    y = [inv_cdf(X) for X in x]
    return y

In [ ]:
# Use a fixed seed
np.random.seed(22092009)

# Apply the transformation method to each of the numbers in x
t1_start = time.perf_counter()
y = transformationMethod(inv_cdf1, 100000)
t1_end = time.perf_counter()

# Plot
X = np.linspace(0, np.pi, 50)
Y = [pdf1(x) for x in X]
plt.hist(y, 16, edgecolor='black', density=True)
plt.plot(X, Y, label='PDF')
plt.xlabel('Random variate')
plt.ylabel('Proportion')
plt.legend()
plt.show()

# Check if the random variates match the distribution they are drawn from
# Calculate the empirical CDF based on the random variates
(ecdf_x, ecdf_y) = ecdf(y)

# Calculate the theoretical CDF of the probability distribution
T = np.linspace(0, np.pi, 50)
CDF = [cdf1(t) for t in T]

# Plot
plt.plot(ecdf_x, ecdf_y, label='ECDF')
plt.plot(T, CDF, label='CDF', ls='dashed')
plt.xlabel('Random variate')
plt.ylabel('CDF')
plt.grid()
plt.legend()
plt.show()

<p><b>(c)</b> Write another short program to do the same thing instead using the rejection method, for the distribution $$\mathrm{pdf}(x)=\frac{2}{\pi}\cos^2\left(\frac{x}{2}\right)\,,$$ again over $x \in \left[ 0, \pi \right]$. Include a similar plot, with $10^5$ samples based on a single seed. (Hint: the previous question should be helpful in finding a good comparison function.) What is the ratio of time taken to produce the $10^5$ samples in this question, to the time required in the previous question? What would you expect the ratio to be be, theoretically, and why?</p>

In [ ]:
# Define the PDF and CDF for Q5 (c)
def pdf2(x):
    return 2 * np.square(np.cos(x / 2)) / np.pi

def cdf2(x):
    return (x + np.sin(x)) / np.pi

def rejectionMethod(pdf_target, pdf_compare, inv_cdf, M, n=1):
    """
    Generate random variables using the rejection method.

    This function generates random variables from a probability distribution
    given a comparison probability distribution.

    Parameters
    ----------
    pdf_target : function
        A target probability distribution for the generated random variables.
    
    pdf_compare : function
        A known probability distribution that can be used as a comparison.
    
    inv_cdf : function
        An inverse CDF function used to generate random numbers from the
        compare PDF.
    
    M : scalar
        A positive float such that (pdf_target / (M * pdf_compare)) is always
        less than one.
    
    n : int, optional
        The number of random variables to generate, defaults to one.

    Returns
    -------
    y : list
        A list containing the random variables.

    Raises
    ------
    ValueError
        If `pdf_target`, `pdf_compare` or `inv_cdf` are not callable functions.
        
        If `pdf_target`, `pdf_compare` or `inv_cdf` do not return a `float`,
        `np.float32`, `np.float64` or `np.longdouble` object.
        
        If `M` is less than or equal to zero.
        
        If `n` is non-integer or less than one.

    See Also
    --------
    transformationMethod : Generates random variables using the transformation
        method.

    Notes
    -----
    A rejection method uses a comparator PDF to determine if numbers drawn from
    a PDF are distributed according to the required PDF. This method is more
    inefficient than the transformation method and should only be used when an
    analytic approach is not possible.

    References
    ----------
    Numerical Recipes in C, Random numbers

    Examples
    --------    
    Generate a vector of random numbers
    >>> rejectionMethod(pdf2, pdf1, inv_cdf1, 4 / np.pi, 3)
    [0.48379875217500407, 0.934258665712482, 1.4229047790082276]
    
    """
    
    if (n / 1.0).is_integer() == False or n < 1:
        raise ValueError('Argument `n` must be a positive integer.')
    
    if M < eps:
        raise ValueError('Argument `M` is too small.')
    
    # Create a list of float functions which can be passed to the function
    permitted_fns = [float, np.float32, np.float64, np.longdouble]
    
    # Raise ValueError if the argument is not a function
    if (callable(pdf_target) == False or callable(pdf_compare) == False or
        callable(inv_cdf) == False):
        raise ValueError('Arguments `pdf_target`, `pdf_compare` and ' +
                         '`inv_cdf` must be functions.')
    
    # Raise ValueError if the argument does not generate a float
    if (type(pdf_target(1)) not in permitted_fns or
        type(pdf_compare(1)) not in permitted_fns or
        type(inv_cdf(1)) not in permitted_fns):
        raise ValueError('Arguments `pdf_target`, `pdf_compare` and ' +
                         '`inv_cdf` must be a float generator function.')
    
    # Generate a vector of length n using inverse CDF of known PDF
    y = transformationMethod(inv_cdf, n)
    
    # Generate a vector of length n of uniformly distributed numbers
    u = np.random.rand(n)
    
    # Verify that each number generated satisfies the acceptance criteria
    for i in range(len(y)):
        # Continue to reject numbers until acceptance criteria is satisfied
        while u[i] > pdf_target(y[i]) / (M * pdf_compare(y[i])):
            u[i] = np.random.rand(1)[0]
            y[i] = transformationMethod(inv_cdf)[0]
    
    return y

In [ ]:
# Use a fixed seed
np.random.seed(22092009)

# Generate random numbers according to PDF
t2_start = time.perf_counter()
y = rejectionMethod(pdf2, pdf1, inv_cdf1, 4 / np.pi, 100000)
t2_end = time.perf_counter()

# Plot
X = np.linspace(0, np.pi, 50)
Y = [pdf2(x) for x in X]
plt.hist(y, 16, edgecolor='black', density=True)
plt.plot(X, Y, label='PDF')
plt.xlabel('Random variate')
plt.ylabel('Proportion')
plt.legend()
plt.show()

# Check if the random variates match the distribution they are drawn from
# Calculate the empirical CDF based on the random variates
(ecdf_x, ecdf_y) = ecdf(y)

# Calculate the theoretical CDF of the probability distribution
T = np.linspace(0, np.pi, 50)
CDF = [cdf2(t) for t in T]

# Plot
plt.plot(ecdf_x, ecdf_y, label='ECDF')
plt.plot(T, CDF, label='CDF', ls='dashed')
plt.grid()
plt.xlabel('Random variate')
plt.ylabel('CDF')
plt.legend()
plt.show()

delta_t1 = t1_end - t1_start
delta_t2 = t2_end - t2_start
print(("The rejection method is a factor of %.2f slower than the " +
      "transformation method.") % (delta_t2 / delta_t1))

In [ ]:
"""
Rejection method efficiency

"""

X = np.linspace(0, np.pi, 50)
Y1 = [1 for x in X]
Y2 = [np.cos(x / 2) for x in X]

plt.plot(X, Y1, ls='')
plt.plot(X, Y2, label='$\\frac{g}{Mf}$')
plt.fill_between(X, Y1, Y2, color='#e57373')
plt.fill_between(X, Y2, color='#A5D6A7')
plt.ylabel('$u_i$')
plt.xlabel('$y_i$')
plt.legend()
plt.grid()

In [ ]:
"""
We can take this one step further and consider a histogram of how
much faster the transformation method is compared to the rejection
method. To reduce the computation time, we will only generate
1000 numbers in each trial, but we will run the experiment for
1000 trials.

Computation time is approximately 30 secs.

"""

# Randomise seed for this experiment!
np.random.seed()

timeFactors = []
n = 1000
trials = 1000

for i in range(trials):
    # Generate numbers using the transformation method
    t1_start = time.perf_counter()
    y = transformationMethod(inv_cdf1, n)
    t1_end = time.perf_counter()
    
    # Generate numbers using the rejection method
    t2_start = time.perf_counter()
    y = rejectionMethod(pdf2, pdf1, inv_cdf1, 4 / np.pi, n)
    t2_end = time.perf_counter()
    
    delta_t1 = t1_end - t1_start
    delta_t2 = t2_end - t2_start
    timeFactors.append(delta_t2 / delta_t1)

plt.hist(timeFactors, edgecolor='black', density=True)
plt.xlabel('$\delta$')
plt.ylabel('Proportion')
plt.show()

print('Mean: ', np.mean(timeFactors))
print('Standard deviation: ', np.std(timeFactors))